# Comparing Corpora

The lecture talked about domain-specific languages or sub-languages. 
This manifests in various aspects of language like syntax, vocabulary or phrases.

This exercise tries to confirm this by comparing subcorpora in terms of frequencies.

The vocabulary of a corpus can be estimated by frequencies. Of course there are words that are so frequent that they occurr in every corpus and do not manifest a sublanguage and are often called 'stopwords'. To establish a domain specific vocabulary it does make sense to only count ngrams of words that do not contain a stopword.


In [ ]:
#%pip install pandas
import nltk
import string
from nltk.corpus import stopwords
import pandas as pd
pd.set_option('display.max_rows', 500)

Download the brown corpus.

_"The Brown Corpus was the first million-word electronic corpus of English, created in 1961 at Brown University. This corpus contains text from 500 sources, and the sources have been categorized by genre, such as news, editorial"_

_--_ ( https://www.nltk.org/book/ch02.html ) 


To compare the vocabulary and phrases  of each corpus with each other, count ngram frequencies for every category. NLTK provides already pre-tokenized and cleaned text for this. Use `nltk.corpus.brown.words(category=...)` to load the corpus.

The approach is to count frequencies in each corpus and compare the top k of each corpus with each other to spot differences. This allows a rather explorative view on the concept of sublanguage.

In [ ]:
nltk.download("brown")

In [ ]:
nltk.corpus.brown.categories()

In [ ]:
genres = ["news", "religion", "fiction", "humor"]
texts = {genre: list(nltk.corpus.brown.words(categories=genre)) for genre in genres}
for genre, text in texts.items():
    print(genre+":",text[:50],"\n")

# Exercise

Compare the corpora in terms of 
    - Type Token Ratio 
    - Vocabulary (most frequent words or multi-word units that do not contain stopwords)
    - Most Frequent syntactic structures (POS-tag sequences) 

## Type Token Ratio

In [ ]:
def ttr(text):
    tokens = [x.lower() for x in text]
    types = set(tokens)
    return 100 * len(types) / len(tokens)

In [ ]:
import pandas as pd
def print_ttr(texts):
    """Function to print the type token ratios."""
    ttrs = [(k, ttr(v)) for k,v in texts.items()]
    return pd.DataFrame(ttrs)

print_ttr(texts)


## Print gram frequencies

In [ ]:
text = texts["humor"]
text = [x.lower() for x in text]
text = [x for x in nltk.ngrams(text, 2)]
print(text[:10])
stopwords = set(nltk.corpus.stopwords.words("english")) | set(string.punctuation) | {"``", '"', "''", "--"}
#print(stopwords)
text = [x for x in text if all(w.strip() not in stopwords for w in x)] 
print(text[:10])

In [ ]:
def preprocess(text, ngram):
    """Function to preprocess the text and generate ngrams"""
    text = [x.lower() for x in text]
    text = [x for x in nltk.ngrams(text, ngram)]   
    
    stopwords = set(nltk.corpus.stopwords.words("english")) | set(string.punctuation) | {"``", '"', "''", "--"}
    text = [x for x in text if all(w.strip() not in stopwords for w in x)]    
        
    return text

In [ ]:
def print_frequencies(texts, n, topk=100):
    """Function to count and print the topk ngrams."""
    word_frequencies = {k:nltk.FreqDist(preprocess(v, n)) for k,v in texts.items()}
    compare = {k: list(word_frequencies[k])[:topk] for k in texts.keys()}
    return pd.DataFrame.from_dict(compare)

In [ ]:
print_frequencies(texts, 1, topk=50)

In [ ]:
print_frequencies(texts, 2, topk=20)

In [ ]:
print_frequencies(texts, 3, topk=50)

In [ ]:
print_frequencies(texts, 4, topk=50)

## Pos Patterns

In [ ]:
def print_pattern_frequencies(texts, n, topk=100):
    """Function to count and print the topk ngrams."""
    pos_tags = {k:[x[1] for x in nltk.pos_tag(v)] for k, v in texts.items()}
    pos_tags_ngrams = {k: nltk.ngrams(v, n) for k, v in pos_tags.items()}
    frequencies = {k:nltk.FreqDist(v) for k,v in pos_tags_ngrams.items()}
    compare = {k: list(frequencies[k])[:topk] for k in texts.keys()}
    return pd.DataFrame.from_dict(compare)

In [ ]:
print_pattern_frequencies(texts, n=2, topk=10)

In [ ]:
print_pattern_frequencies(texts, n=3, topk=10)